In [112]:
import pandas as pd
import numpy as np
import math

DATA_PATH = "/Users/mvonebers/HUBBS-Lab/data/"

e4_data = pd.read_excel(DATA_PATH + "E4_TEST.xlsx")

e4_data.insert(0, "person", [0] * e4_data.shape[0])
e4_data.insert(1, "trial", [0] * e4_data.shape[0])

for i, row in e4_data.iterrows():
    e4_data.at[i, "person"] = int(e4_data.at[i, "id"][7:])
    e4_data.at[i, "trial"] = int(e4_data.at[i, "id"][5])
    
e4_data = e4_data.drop(columns=['id'])
e4_data = e4_data.rename(columns={"person": "id"})

change_data = pd.read_excel(DATA_PATH + "normalized_change.xlsx")

e4_data_results = pd.merge(e4_data, change_data, on='id')
e4_data_results

ids = set()
for i in range(e4_data.shape[0]):
    ids.add(e4_data.iloc[i, 0])
    
#print(len(ids))
e4_data_results

,id,trial,EDA_PPT,HR_PPT,TEMP_PPT,BVP_PPT,ACC_PPT,IBI_PPT,EDA_FREQ_PPT,EDA_AMP_PPT,Brief fear of Negative Evaluation,CAI Trait Dyadic Score,CAI Trait Full Score,CAI Trait Public Speaking Score,CAI Trait Small group Score,STAI Trait Score,CAI State Score,STAI State Score
0,4,1,0.205970,82.376250,31.404085,0.013278,64.812218,0.882853,7.181628,0.012059,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
1,4,2,0.206435,105.666378,31.331652,-0.132812,65.118669,0.859414,7.312119,0.012054,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
2,4,3,0.212655,75.010328,30.183614,-0.111220,64.876459,0.921917,7.685270,0.012726,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
3,4,4,0.191278,92.559115,29.766569,-0.046831,64.844329,1.171929,6.262231,0.010277,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
4,4,5,0.154794,79.935382,29.652207,0.029759,62.817997,1.187555,2.574713,0.010306,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,73,4,2.422543,78.467647,32.132378,-0.054789,65.853056,0.859414,8.862629,0.031206,0.344828,0.125000,0.233333,0.307692,0.222222,0.033333,0.178571,0.000000
156,73,5,0.545086,78.157361,33.548290,-0.043703,65.637811,0.655458,10.471204,0.026741,0.344828,0.125000,0.233333,0.307692,0.222222,0.033333,0.178571,0.000000
157,73,6,0.840680,84.307227,33.130568,-0.046631,65.968021,0.661489,15.984329,0.044909,0.344828,0.125000,0.233333,0.307692,0.222222,0.033333,0.178571,0.000000
158,73,7,0.997170,85.801776,32.511091,-0.058622,66.282949,0.953169,14.677686,0.052215,0.344828,0.125000,0.233333,0.307692,0.222222,0.033333,0.178571,0.000000


In [2]:
"""
Collect all possible slopes (trial 8 - trial 1)
"""
col = ["id", "trial", "EDA_PPT", "HR_PPT", "TEMP_PPT", "BVP_PPT", "ACC_PPT", 
           "IBI_PPT", "EDA_FREQ_PPT", "EDA_AMP_PPT", "Brief fear of Negative Evaluation",
           "CAI Trait Dyadic Score", "CAI Trait Full Score", "CAI Trait Public Speaking Score", 
           "CAI Trait Small group Score", "STAI Trait Score", "CAI State Score", "STAI State Score"]

slopes = pd.DataFrame(np.zeros((160, 18)), columns=col)
slopes = slopes.drop(["trial"], axis=1)
    
first = [0] * 8
for i in range(e4_data_results.shape[0]):
    if e4_data_results.iloc[i, 1] == 1:
        for j in range(2,10):
            first[j - 2] = e4_data_results.iloc[i, j]
    elif e4_data_results.iloc[i, 1] == 8:
        for j in range(2,10):
            #print(i, j)
            slopes.iloc[i, 0] = e4_data_results.iloc[i, 0]
            for k in range(9, 17):
                slopes.iloc[i, k] = e4_data_results.iloc[i, k + 1]
            if not math.isnan(e4_data_results.iloc[i, j]) and not math.isnan(first[j - 2]):
                slopes.iloc[i,j - 1] = e4_data_results.iloc[i, j] - first[j - 2] / 8
            else:
                slopes.iloc[i, j - 1] = 0
            
index_names = slopes[ slopes['id'] == 0 ].index
slopes.drop(index_names, inplace=True)

slopes.replace(0, np.NaN, inplace=True)
slopes

,id,EDA_PPT,HR_PPT,TEMP_PPT,BVP_PPT,ACC_PPT,IBI_PPT,EDA_FREQ_PPT,EDA_AMP_PPT,Brief fear of Negative Evaluation,CAI Trait Dyadic Score,CAI Trait Full Score,CAI Trait Public Speaking Score,CAI Trait Small group Score,STAI Trait Score,CAI State Score,STAI State Score
7,4.0,0.210164,60.664894,23.554450,0.037829,55.088773,0.697716,0.222919,0.006630,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
15,5.0,0.444626,63.212511,25.405799,-0.055597,57.430220,0.614378,5.373883,0.073715,0.388889,0.166667,0.121212,0.100000,0.090909,0.025641,0.300000,0.090909
23,8.0,0.381983,72.302272,29.130058,-0.075223,57.577599,0.724642,3.257187,0.009408,0.089286,0.133333,0.128205,0.166667,0.083333,0.155172,0.187500,0.068182
31,20.0,-0.103883,85.766964,27.464529,-0.020019,56.391233,0.953169,14.729787,0.005928,0.272727,NaN,0.043478,0.125000,NaN,0.040000,-0.037037,-0.047619
39,21.0,-0.095482,78.963769,29.065740,0.017702,55.473673,0.445426,-0.779171,-0.045754,0.117647,0.312500,0.121212,-0.111111,NaN,0.114286,0.380952,0.512195
47,23.0,0.117118,73.496043,25.305126,-0.010327,55.741410,0.917453,4.563054,0.005931,-0.111111,NaN,0.066667,0.250000,-0.058824,0.100000,0.205128,0.230769
55,32.0,-0.088614,58.701567,28.561196,0.023527,55.746192,NaN,-1.675824,-0.004815,0.486486,0.375000,0.272727,0.470588,-0.181818,-0.214286,0.274510,0.125000
63,35.0,1.632557,77.821916,29.437755,0.073159,56.364890,0.651071,9.164304,0.158474,0.333333,-0.142857,NaN,NaN,0.125000,-0.068966,0.258065,0.050000
71,37.0,0.032494,80.075242,23.888878,0.084808,55.412272,NaN,1.518812,0.005851,-0.551724,0.176471,0.025641,-0.100000,-0.083333,-0.107143,0.302326,0.045455
79,41.0,0.277439,66.333266,27.803641,0.003895,57.982554,NaN,3.064312,0.007803,0.200000,0.217391,0.138462,0.136364,0.050000,NaN,0.263158,-0.555556


In [3]:
import matplotlib.pyplot as plt

# Fixing random state for reproducibility
np.random.seed(42)


N = 50
x_eda = slopes['EDA_PPT'].to_numpy(copy=True)
x_hr = slopes['HR_PPT'].to_numpy(copy=True)
x_temp = slopes['TEMP_PPT'].to_numpy(copy=True)
x_bvp = slopes['BVP_PPT'].to_numpy(copy=True)
x_acc = slopes['ACC_PPT'].to_numpy(copy=True)
x_ibi = slopes['IBI_PPT'].to_numpy(copy=True)
y = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)

f1 = plt.figure(1)
plt.scatter(x_eda, y, alpha=0.5)
f2 = plt.figure(2)
plt.scatter(x_hr, y, alpha=0.5)
f3 = plt.figure(3)
plt.scatter(x_temp, y, alpha=0.5)
f4 = plt.figure(4)
plt.scatter(x_bvp, y, alpha=0.5)
f5 = plt.figure(5)
plt.scatter(x_acc, y, alpha=0.5)
f6 = plt.figure(6)
plt.scatter(x_i)

plt.show()

NameError: name 'x_i' is not defined

In [199]:
# start at 60-40 and if it's inconsistent, then slowly increase slowly towards 80-20 but don't go past that

# Modified for Oct. 21st checkpoint to collect scores and put in a table

# Oct. 28th checkpoint get p-values

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy import stats

Y_BFNE = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)

k = 0

predicted = slopes.copy(deep=True)

for col in slopes.columns:
    if col == 'id':
        continue
    if col == 'Brief fear of Negative Evaluation':
        break
    print(col)
    X_before = slopes[col].to_numpy(copy=True)
    Y_before = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)
    X = np.array([])
    Y = np.array([])
    for i in range(len(X_before)):  # remove NaN from data
        if not math.isnan(X_before[i]) and not math.isnan(Y_before[i]):
            X = np.append(X, X_before[i])
            Y = np.append(Y, Y_before[i])
    
    X = X.reshape(-1, 1)
    lm = LinearRegression()
    lm.fit(X,Y)
    params = np.append(lm.intercept_,lm.coef_)
    predictions = lm.predict(X)

    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((Y-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    myDF3 = pd.DataFrame()
    myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]
    print(myDF3)
     
        
"""model = LinearRegression()
    cv = KFold(10, shuffle=True, random_state=41)
    predicts = cross_val_predict(model, X.reshape(-1,1), Y, cv=cv)
    
    for _, test in cv.split(X):
        for pred, test_index in zip(predicts, test):
            predicted.at[test_index,col] = pred
            
# do this for all of the traits not just BFNE

#correlations_BFNE = []

print("---------- Correlations ----------")   
for j in range(9, len(predicted.columns)):
    Y_0 = np.array(predicted[predicted.columns[j]][:20])
    for i in range(1, 9):
        print(predicted.columns[i], "vs." , predicted.columns[j], ":")
        X_0 = np.array(predicted[predicted.columns[i]][:20])
        X = []
        Y = []
        for i in range(len(X_0)):
            if not (math.isnan(X_0[i]) or math.isnan(Y_0[i])):
                X.append(X_0[i])
                Y.append(Y_0[i])

        correlation = np.corrcoef(X, Y)[0][1]
        #print(correlation)
        #correlations_BFNE.append(correlation)"""
    
#predicted
#plt.show()
# some notable correlations:
# * EDA_PPT vs. CAI Trait Full Score : 0.23216250036705188
# * EDA_FREQ_PPT vs. CAI Trait Full Score : 0.18947315542233634
# * EDA_FREQ_PPT vs. CAI Trait Public Speaking Score : 0.21350034134581444
# * EDA_PPT vs. CAI Trait Small group Score : 0.590551086671041
# * EDA_FREQ_PPT vs. CAI Trait Small group Score : 0.6218508358580933
# * EDA_AMP_PPT vs. CAI Trait Small group Score : 0.3619892407906545
# * EDA_PPT vs. STAI Trait Score : 0.31234968134619207
# * IBI_PPT vs. STAI Trait Score : 0.33955692294262746

# get significant values from correlations
#try combinations like acoustics, etc and then all together

EDA_PPT
   Coefficients  Standard Errors  t values  Probabilites
0        0.1056            0.057     1.842         0.081
1        0.0718            0.102     0.706         0.489
HR_PPT
   Coefficients  Standard Errors  t values  Probabilites
0        0.5970            0.316     1.892         0.074
1       -0.0063            0.004    -1.504         0.149
TEMP_PPT
   Coefficients  Standard Errors  t values  Probabilites
0       -1.3883            0.686    -2.025         0.057
1        0.0564            0.025     2.215         0.039
BVP_PPT
   Coefficients  Standard Errors  t values  Probabilites
0        0.1265            0.049     2.585         0.018
1       -0.2414            0.664    -0.364         0.720
ACC_PPT
   Coefficients  Standard Errors  t values  Probabilites
0       -4.6971             2.79    -1.683         0.109
1        0.0859             0.05     1.729         0.100
IBI_PPT
   Coefficients  Standard Errors  t values  Probabilites
0        0.2221            0.166     1.3

'model = LinearRegression()\n    cv = KFold(10, shuffle=True, random_state=41)\n    predicts = cross_val_predict(model, X.reshape(-1,1), Y, cv=cv)\n    \n    for _, test in cv.split(X):\n        for pred, test_index in zip(predicts, test):\n            predicted.at[test_index,col] = pred\n            \n# do this for all of the traits not just BFNE\n\n#correlations_BFNE = []\n\nprint("---------- Correlations ----------")   \nfor j in range(9, len(predicted.columns)):\n    Y_0 = np.array(predicted[predicted.columns[j]][:20])\n    for i in range(1, 9):\n        print(predicted.columns[i], "vs." , predicted.columns[j], ":")\n        X_0 = np.array(predicted[predicted.columns[i]][:20])\n        X = []\n        Y = []\n        for i in range(len(X_0)):\n            if not (math.isnan(X_0[i]) or math.isnan(Y_0[i])):\n                X.append(X_0[i])\n                Y.append(Y_0[i])\n\n        correlation = np.corrcoef(X, Y)[0][1]\n        #print(correlation)\n        #correlations_BFNE.appen

In [4]:
# Trying again with just the raw data instead of the slopes...not sure how that's gonna go
k = 0



for col in e4_data_results.columns:
    if col == 'id' or col == 'trial':
        continue
    if col == 'Brief fear of Negative Evaluation':
        break
    print(col)
    X_before = e4_data_results[col].to_numpy(copy=True)
    Y_before = e4_data_results['Brief fear of Negative Evaluation'].to_numpy(copy=True)
    X = np.array([])
    Y = np.array([])
    for i in range(len(X_before)):  # remove NaN from data
        if not math.isnan(X_before[i]) and not math.isnan(Y_before[i]):
            X = np.append(X, X_before[i])
            Y = np.append(Y, Y_before[i])
            
    #for j in range(50, 10, -2):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=25)
    reg = LinearRegression().fit(X_train.reshape(-1, 1), Y_train.reshape(-1, 1))
    print("Testing with",j,"%", reg.score(X_test.reshape(-1, 1), Y_test.reshape(-1, 1)))
    pred = reg.predict(X_test.reshape(-1,1))
    f1 = plt.figure(k)
    plt.scatter(X, Y,  color='gray')
    plt.plot(X_test, pred, color='red', linewidth=2)
    
    k += 1
    
plt.show()

EDA_PPT
Testing with 16 % -0.5151532081393353


NameError: name 'plt' is not defined

In [5]:
# To do for Monday, Oct. 7th
# cross validation with 10% - done above for slopes
# collect predicted values for all tests and compute correlation vs those and actual values
# trials as seperate features
# feed all 8 trials as feature

Y = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)

columns = list(e4_data_results.columns.values)

for i in range(2, 10):
    # Collect X for each column, with each trial as a feature
    X_0 = []
    X_row = []
    for j in range(e4_data_results.shape[0]):
        if math.isnan(e4_data_results.iloc[j, i]): # if NaN, take another value from a different trial
            X_row.append(0)
        else:
            X_row.append(e4_data_results.iloc[j, i])

        if e4_data_results.at[j, "trial"] == 8:
            X_0.append(X_row)
            X_row = []
            
    X = np.array(X_0)
    print(X[0], Y[0])
    
    model = LinearRegression()
    cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
    scores = cross_val_score(model, X, Y, cv=cv)
    print(columns[i], scores)
    print("Average:", np.average(scores))
    print()
    #X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=25)
    #reg = LinearRegression().fit(X_train, Y_train)
    #print(columns[i], reg.score(X_test, Y_test))
    
    # use correlation instead of default scorer

[0.20596962 0.20643509 0.21265476 0.19127845 0.15479406 0.13110307
 0.22114062 0.23591062] 0.1379310344827586
EDA_PPT [-4.78717319e+00 -1.14991510e+01 -3.12822391e+05 -4.61307119e-01
 -2.52077168e+03 -2.49772314e+00 -2.35478425e+00 -2.91708543e+00
 -4.41419664e+01 -9.81991926e-01]
Average: -31541.28041921626

[ 82.37625    105.66637838  75.01032847  92.55911458  79.93538226
  69.97633929  72.4075841   70.96192547] 0.1379310344827586
HR_PPT [-3.58163435e+00 -3.48619134e-01 -1.40290688e+03 -2.17064234e+01
 -8.57436759e+01  2.89674959e-01 -2.94733087e+02 -4.52753311e+01
 -6.17037855e+01 -6.67941064e+02]
Average: -258.3650822169281

[31.4040849  31.331652   30.18361391 29.76656882 29.6522069  28.4896868
 27.77996169 27.47996109] 0.1379310344827586
TEMP_PPT [   -6.89055035    -1.59697804  -192.40249098  -170.69637107
   -48.85897294    -1.58421488 -1219.99560646   -57.16009086
   -29.04802256  -453.8790691 ]
Average: -218.2112367248038

[ 0.0132785  -0.13281238 -0.1112196  -0.04683107  0.02

In [6]:
for i in range(2, 10):
    # Collect X for each column, with each trial as a feature
    X_0 = []
    X_row = []
    for j in range(e4_data_results.shape[0]):
        if math.isnan(e4_data_results.iloc[j, i]): # if NaN, take another value from a different trial
            X_row.append(0)
        else:
            X_row.append(e4_data_results.iloc[j, i])

        if e4_data_results.at[j, "trial"] == 8:
            X_0.append(X_row)
            X_row = []
            
    X = np.array(X_0)
    
    model = LinearRegression()
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=50)
    reg = model.fit(X_train, Y_train)
    predict = reg.predict(X_test)
    
    print(columns[i], np.corrcoef(predict, Y_test)[0][1])

EDA_PPT -0.04343226185486706
HR_PPT 0.14461114267094322
TEMP_PPT 0.7195961428514247
BVP_PPT 0.19465834650483094
ACC_PPT 0.16458540101912236
IBI_PPT -0.4765407205002237
EDA_FREQ_PPT -0.5571901345414663
EDA_AMP_PPT -0.18481119423451495


In [7]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

#Y = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)

columns = list(e4_data_results.columns.values)

def scorer(y, y_p):
    return np.corrcoef(y_p, y)[1][0]

for index in range(10, len(columns)):
    Y_1 = slopes[columns[index]].to_numpy(copy=True)
    print("-------------", columns[index], "-------------")
    for i in range(2, 10):
        # Collect X for each column, with each trial as a feature
        X_0 = []
        X_row = []
        for j in range(e4_data_results.shape[0]):
            if math.isnan(e4_data_results.iloc[j, i]): # if NaN, take another value from a different trial
                X_row.append(0)
            else:
                X_row.append(e4_data_results.iloc[j, i])

            if e4_data_results.at[j, "trial"] == 8:
                X_0.append(X_row)
                X_row = []

        X_1 = np.array(X_0)
        X = []
        Y = []
        for k in range(len(X_1)):
            if not math.isnan(Y_1[k]):
                X.append(X_1[k])
                Y.append(Y_1[k])
                
        #print(X_1)
        model = LinearRegression()
        cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
        scores = cross_val_score(model, X, Y, scoring=make_scorer(scorer), cv=cv)
        print(columns[i], scores)
        print("Average:", np.average(scores))
        print("Standard dev:", np.std(scores))
        print()
        #X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=25)
        #reg = LinearRegression().fit(X_train, Y_train)
        #print(columns[i], reg.score(X_test, Y_test))
    
    
    # use correlation instead of default scorer

------------- Brief fear of Negative Evaluation -------------
EDA_PPT [-0.83487281 -0.90808068  0.18868763 -0.13617651  0.29105022  0.01169638
 -0.01577298  0.41823595  0.4338947   0.56443246]
Average: 0.0013094366789602185
Standard dev: 0.48400607338573787

HR_PPT [ 0.5663172   0.59915334 -0.79100374  0.86094802 -0.65346069  0.88051504
  0.90932018 -0.85818658 -0.31616814 -0.34758497]
Average: 0.08498496530099847
Standard dev: 0.7043320573884961

TEMP_PPT [ 0.76172817 -0.24436965 -0.53533729 -0.99365183 -0.1526365   0.90797731
 -0.26419968  0.42550315  0.62337802 -0.72284078]
Average: -0.01944490923367075
Standard dev: 0.6260806095643807

BVP_PPT [-0.93521855  0.00718266  0.06312498  0.27168741  0.92903154  0.58124525
 -0.30849613  0.41958678 -0.16068318  0.38634686]
Average: 0.12538076219351443
Standard dev: 0.4942198243175945

ACC_PPT [ 0.55048674 -0.05927025 -0.68667037 -0.99094834  0.0618513  -0.48280083
 -0.72493436 -0.16247825  0.38976055  0.16970296]
Average: -0.193530085797591

HR_PPT [ 0.68945214  0.7834099  -0.55846269  0.31086757  0.87791085  0.2522737
  0.78061452 -0.52428785  0.65223959  0.66638418]
Average: 0.39304019042290794
Standard dev: 0.5040713482222828

TEMP_PPT [-0.44893126 -0.68918378  0.73846538  0.13241331 -0.5507969  -0.76655709
 -0.41344544 -0.71113703 -0.24846856 -0.57174301]
Average: -0.3529384369968681
Standard dev: 0.44158682573108043

BVP_PPT [-0.88921507 -0.96797352 -0.64879988 -0.76773351 -0.10320837  0.61716014
 -0.95400158 -0.90617175 -0.39438169 -0.07972241]
Average: -0.5094047647005502
Standard dev: 0.4927268526628789

ACC_PPT [-0.46911936  0.02995019  0.81818059  0.23437891 -0.25685206  0.61689437
 -0.96842593  0.48293713 -0.29236095  0.80659368]
Average: 0.10021765676199419
Standard dev: 0.5655584954805727

IBI_PPT [-0.88596991 -0.97374282 -0.70070304 -0.83689702 -0.28207836 -0.64326937
 -0.96000196  0.31450822 -0.35206858 -0.82023687]
Average: -0.6140459706609736
Standard dev: 0.38270070133025846

EDA_FREQ_PPT [-0.42050959 -0.

In [192]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

X = []

correlations = []

for index, row in slopes.iterrows():
    my_list = [row.EDA_PPT, row.HR_PPT, row.TEMP_PPT, row.BVP_PPT, row.ACC_PPT, row.IBI_PPT, row.EDA_FREQ_PPT, row.EDA_AMP_PPT]
    X.append(my_list)
        

for index in range(10, len(columns)):
    Y_1 = slopes[columns[index]].to_numpy(copy=True)
    print("-------------", columns[index], "-------------")  
    Y = []
    X = []
    for i in range(len(Y_1)):
        if not math.isnan(Y_1[i]):
            isnan = False
            for j in range(len(X_1[i])):
                if math.isnan(X_1[i][j]):
                    isnan = True
                    break
            if not isnan:
                Y.append(Y_1[i])
                X.append(X_1[i])
    
    model = LinearRegression()
    cv = ShuffleSplit(n_splits=20, test_size=0.2, random_state=0)
    #scores = cross_val_score(model, X, Y, scoring=make_scorer(scorer), cv=cv)
    
    corr = []
    for score in scores:
        corr.append(score)
    correlations.append(corr)    
    
    print(scores)
    print("Average:", np.average(scores))
    print("Standard dev:", np.std(scores))
    print()
    
# "Correlations" variable holds the scores for each set of features (i.e. each row from table "slopes") 
# for each of the Y sets

# did they want the score for each individual slope instead?

------------- Brief fear of Negative Evaluation -------------
[ 0.18124326  0.87410605  0.87271241 -0.42839825  0.24751528  0.84611155
  0.5951216   0.48408284  0.22758281  0.46564316]
Average: 0.4365720715188557
Standard dev: 0.383458309296573

------------- CAI Trait Dyadic Score -------------
[ 0.18124326  0.87410605  0.87271241 -0.42839825  0.24751528  0.84611155
  0.5951216   0.48408284  0.22758281  0.46564316]
Average: 0.4365720715188557
Standard dev: 0.383458309296573

------------- CAI Trait Full Score -------------
[ 0.18124326  0.87410605  0.87271241 -0.42839825  0.24751528  0.84611155
  0.5951216   0.48408284  0.22758281  0.46564316]
Average: 0.4365720715188557
Standard dev: 0.383458309296573

------------- CAI Trait Public Speaking Score -------------
[ 0.18124326  0.87410605  0.87271241 -0.42839825  0.24751528  0.84611155
  0.5951216   0.48408284  0.22758281  0.46564316]
Average: 0.4365720715188557
Standard dev: 0.383458309296573

------------- CAI Trait Small group Score 

In [201]:
# get significant values from correlations
#try combinations like acoustics, etc and then all together
from copy import deepcopy

"""
import statsmodels.api as sm

mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']
"""

DATA_PATH = "/Users/mvonebers/HUBBS-Lab/data/"
audio_data = pd.read_excel(DATA_PATH + "audio_TEST.xlsx")

audio_data.insert(0, "person", [0] * audio_data.shape[0])
audio_data.insert(1, "trial", [0] * audio_data.shape[0])

for i, row in audio_data.iterrows():
    audio_data.at[i, "person"] = int(audio_data.at[i, "id"][7:])
    audio_data.at[i, "trial"] = int(audio_data.at[i, "id"][5])
    
audio_data = audio_data.drop(columns=['id'])
audio_data = audio_data.rename(columns={"person": "id"})
#audio_data.head(20)
audio_data = audio_data[audio_data.id != 16]
audio_data = audio_data[audio_data.id != 38]
audio_data = audio_data[audio_data.id != 46]
audio_data = audio_data[audio_data.id != 53]
audio_data = audio_data[audio_data.id != 58]
audio_data = audio_data[audio_data.id != 65]
e4_data_results = e4_data_results[e4_data_results.id != 66]

for col in e4_data_results.columns[1:]:
    audio_data[col] = e4_data_results[col]
    
combined = deepcopy(audio_data)
combined = combined[combined.id != 66]
    
combined.tail(50)
#combined = pd.merge(audio_data, e4_data_results, on='id')
#combined.head(30)
#combined = combined.drop(columns=['trial_y'])
#combined = combined.rename(columns={"trial_x": "trial"})

#combined.head(20)

,id,trial,pcm_RMSenergy_sma_amean,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_amean,...,EDA_FREQ_PPT,EDA_AMP_PPT,Brief fear of Negative Evaluation,CAI Trait Dyadic Score,CAI Trait Full Score,CAI Trait Public Speaking Score,CAI Trait Small group Score,STAI Trait Score,CAI State Score,STAI State Score
126,47,7.0,0.019391,-7.435403,-6.047399,-2.233446,-10.169961,4.926539,-19.460316,-4.208723,...,10.383189,0.023702,0.058824,0.266667,0.173913,-0.0625,0.333333,0.395349,0.222222,0.500000
127,47,8.0,0.018885,-5.278236,-5.775854,-1.615624,-9.734565,3.670642,-18.946023,-5.272104,...,14.555712,0.101453,0.058824,0.266667,0.173913,-0.0625,0.333333,0.395349,0.222222,0.500000
128,50,1.0,0.017626,-4.704711,-0.718251,-0.080680,-13.133022,7.700459,-15.449754,-9.233024,...,8.535140,0.036159,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
129,50,2.0,0.017809,-4.479853,0.536077,1.094122,-15.146089,6.592047,-13.610139,-6.570396,...,5.798658,0.040306,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
130,50,3.0,0.024452,-4.649292,-0.624613,1.675421,-17.683803,8.014135,-15.113263,-6.385127,...,4.494382,0.026497,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
131,50,4.0,0.024528,-7.081521,-0.608256,1.238764,-17.848186,6.906087,-11.836202,-7.565406,...,8.074012,0.021147,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
132,50,5.0,0.023953,-4.696744,0.202060,1.420300,-19.446912,9.759478,-12.998799,-9.055439,...,1.577218,0.012702,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
133,50,6.0,0.023686,-5.054646,-0.193796,3.177377,-17.415487,4.800906,-10.808127,-6.855264,...,2.289679,0.011130,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
134,50,7.0,0.020003,-5.157085,-0.845543,0.485564,-16.355466,7.102649,-10.928773,-7.171985,...,3.553362,0.018580,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048
135,50,8.0,0.019474,-2.968418,-0.460177,-0.841534,-15.776103,7.136250,-12.395238,-7.690178,...,4.367246,0.011727,0.108696,0.263158,0.098361,0.0000,0.045455,0.169231,0.040000,-0.119048


In [202]:
# Collect slopes
combined_slopes = pd.DataFrame(np.zeros((combined.shape[0], combined.shape[1])), columns=combined.columns)

first = [0] * combined.shape[1]
for i in range(combined.shape[0]):
    if combined.iloc[i, 1] == 1:
        for j in range(2,34):
            first[j - 2] = combined.iloc[i, j]
    elif combined.iloc[i, 1] == 8:
        for j in range(2,34):
            #print(i, j)
            combined_slopes.iloc[i, 0] = combined.iloc[i, 0] # copy ID over
            for k in range(33, 42):
                #print(k)
                combined_slopes.iloc[i, k] = combined.iloc[i, k]
            #if not math.isnan(combined.iloc[i, j]) and not math.isnan(first[j - 2]):
                #print(combined.columns[j], combined.iloc[i,j], '-', first[j - 2])
            combined_slopes.iloc[i,j] = combined.iloc[i, j] - first[j - 2] / 8
            #else:
             #   combined_slopes.iloc[i, j] = 0
            
index_names = combined_slopes[combined_slopes['id'] == 0 ].index
combined_slopes.drop(index_names, inplace=True)
combined_slopes = combined_slopes.drop(["trial"], axis=1)

combined_slopes.replace(0, np.NaN, inplace=True)

combined_slopes


,id,pcm_RMSenergy_sma_amean,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_amean,...,EDA_FREQ_PPT,EDA_AMP_PPT,Brief fear of Negative Evaluation,CAI Trait Dyadic Score,CAI Trait Full Score,CAI Trait Public Speaking Score,CAI Trait Small group Score,STAI Trait Score,CAI State Score,STAI State Score
0,4.0,0.001476,-6.397882,-3.530831,-7.468233,-9.191484,-7.239962,-1.519063,-9.185706,4.297587,...,0.222919,0.006630,0.137931,0.153846,0.101695,0.120000,0.047619,-0.068182,-0.179487,-0.047619
1,5.0,0.000183,-3.829912,-3.170033,-3.330801,-5.146229,-4.579870,-5.910732,-5.516717,2.945937,...,5.373883,0.073715,0.388889,0.166667,0.121212,0.100000,0.090909,0.025641,0.300000,0.090909
2,8.0,0.005651,-6.797778,-5.042236,5.585197,-7.190841,-3.646728,-12.304083,-3.311612,4.301590,...,3.257187,0.009408,0.089286,0.133333,0.128205,0.166667,0.083333,0.155172,0.187500,0.068182
3,20.0,0.003536,-2.787734,-11.272312,-1.895499,-10.065208,1.173902,-18.302122,1.451626,1.067014,...,-0.779171,-0.045754,0.117647,0.312500,0.121212,-0.111111,NaN,0.114286,0.380952,0.512195
4,21.0,0.014414,-5.524391,-5.207538,-6.781840,-6.291179,1.168361,-19.842839,1.008793,2.640164,...,4.563054,0.005931,-0.111111,NaN,0.066667,0.250000,-0.058824,0.100000,0.205128,0.230769
5,23.0,-0.000123,-4.977028,-3.039470,-5.884622,-1.562857,-9.792811,-2.300735,-4.205353,0.454194,...,-1.675824,-0.004815,0.486486,0.375000,0.272727,0.470588,-0.181818,-0.214286,0.274510,0.125000
6,32.0,0.003727,0.448182,-2.568618,1.981291,-13.857958,10.793914,-10.185338,-5.354570,2.347244,...,9.164304,0.158474,0.333333,-0.142857,NaN,NaN,0.125000,-0.068966,0.258065,0.050000
7,35.0,0.007008,-5.116303,-3.446236,-4.381496,-7.027117,7.121191,-16.310352,-1.091835,-0.376211,...,1.518812,0.005851,-0.551724,0.176471,0.025641,-0.100000,-0.083333,-0.107143,0.302326,0.045455
8,37.0,0.003931,-2.315065,-3.043218,-9.334525,-7.577881,0.750757,-16.371492,-1.513022,0.058364,...,3.064312,0.007803,0.200000,0.217391,0.138462,0.136364,0.050000,NaN,0.263158,-0.555556
9,41.0,0.005264,-1.441041,-0.364782,2.832470,-6.905629,-10.587943,-6.731009,-7.040643,-5.493998,...,0.709010,NaN,0.109091,-0.333333,-0.325000,-0.266667,-0.400000,0.360000,0.305085,0.510638


In [ ]:
Y_BFNE = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)

k = 0

predicted = slopes.copy(deep=True)

for col in slopes.columns:
    if col == 'id':
        continue
    if col == 'Brief fear of Negative Evaluation':
        break
    #print(col)
    X_before = slopes[col].to_numpy(copy=True)
    Y_before = slopes['Brief fear of Negative Evaluation'].to_numpy(copy=True)
    X = np.array([])
    Y = np.array([])
    for i in range(len(X_before)):  # remove NaN from data
        if not math.isnan(X_before[i]) and not math.isnan(Y_before[i]):
            X = np.append(X, X_before[i])
            Y = np.append(Y, Y_before[i])
     
    model = LinearRegression()
    cv = KFold(10, shuffle=True, random_state=41)
    predicts = cross_val_predict(model, X.reshape(-1,1), Y, cv=cv)
    
    for _, test in cv.split(X):
        for pred, test_index in zip(predicts, test):
            predicted.at[test_index,col] = pred
            
# do this for all of the traits not just BFNE

#correlations_BFNE = []

print("---------- Correlations ----------")   
for j in range(9, len(predicted.columns)):
    Y_0 = np.array(predicted[predicted.columns[j]][:20])
    for i in range(1, 9):
        print(predicted.columns[i], "vs." , predicted.columns[j], ":")
        X_0 = np.array(predicted[predicted.columns[i]][:20])
        X = []
        Y = []
        for i in range(len(X_0)):
            if not (math.isnan(X_0[i]) or math.isnan(Y_0[i])):
                X.append(X_0[i])
                Y.append(Y_0[i])

        correlation = np.corrcoef(X, Y)[0][1]
        if correlation >= 0.3:
            print(correlation)